In [4]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from scipy.stats import zscore

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [5]:
sp500_train = pd.read_csv('Datasets/sp500_train.csv', index_col='Date', parse_dates=True)
sp500_test = pd.read_csv('Datasets/sp500_test.csv', index_col='Date', parse_dates=True)
tesla_train = pd.read_csv('Datasets/tesla_train.csv', index_col ='Date', parse_dates=True)
tesla_test = pd.read_csv('Datasets/tesla_test.csv', index_col='Date', parse_dates=True)

sp500_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 418 entries, 2015-01-01 to 2022-12-29
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   418 non-null    float64
dtypes: float64(1)
memory usage: 6.5 KB


In [8]:
# Preprocess the data for the Prophet model
sp500_train = sp500_train.reset_index()
sp500_test = sp500_test.reset_index()
sp500_train.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)
sp500_test.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)

tesla_train = tesla_train.reset_index()
tesla_test = tesla_test.reset_index()
tesla_train.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)
tesla_test.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)

In [7]:
model = Prophet(weekly_seasonality=True, yearly_seasonality=True)
model.fit(sp500_train)


ValueError: Dataframe must have columns "ds" and "y" with the dates and values respectively.